In [1]:
# Importando Bibliotecas
import requests
import re
import time
import json
from bs4 import BeautifulSoup
import google.generativeai as genai

c:\Users\PedroLustosa\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# URL
diseases_url = "https://www.who.int/news-room/fact-sheets"

# Requisição
try:
    response = requests.get(diseases_url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

diseases_html = BeautifulSoup(response.content, 'html.parser')

In [3]:
# Encontrar todos os elementos com a classe "item"
items = diseases_html.find_all('p')

# Inicializar uma lista para armazenar as doenças
diseases = []

# Exibir os textos dos elementos <a> dentro dos elementos com a classe "item"
for item in items:
        disease = item.text
        diseases.append(disease)



In [4]:
# Acessando a API KEY
with open('dados.json', "r") as arquivo:  # Abre o arquivo 'dados.json' em modo de leitura
    key = json.load(arquivo)  # Carrega o conteúdo do arquivo JSON e o armazena na variável 'key'

# Configurando a chave da API para uso no módulo 'genai'
genai.configure(api_key=key['Gemini'])  # Acessa o valor associado à chave 'Gemini' no dicionário 'key' e configura a API com esta chave


In [5]:
# Configuração de geração de texto
generation_config = {
  "temperature": 1,  # Define a aleatoriedade na geração de texto (1 é um valor padrão para maior variabilidade)
  "response_mime_type": "text/plain",  # Define o tipo MIME da resposta como texto simples
}

# Configurações de segurança para filtrar conteúdo inadequado
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",  # Categoria para assédio
    "threshold": "BLOCK_NONE",  # Não bloquear nenhum conteúdo desta categoria
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",  # Categoria para discurso de ódio
    "threshold": "BLOCK_NONE",  # Não bloquear nenhum conteúdo desta categoria
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",  # Categoria para conteúdo sexual explícito
    "threshold": "BLOCK_NONE",  # Não bloquear nenhum conteúdo desta categoria
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",  # Categoria para conteúdo perigoso
    "threshold": "BLOCK_NONE",  # Não bloquear nenhum conteúdo desta categoria
  },
]

# Inicialização do modelo generativo com as configurações definidas
model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-latest",  # Nome do modelo generativo a ser utilizado
  safety_settings=safety_settings,  # Aplica as configurações de segurança definidas
  generation_config=generation_config,  # Aplica a configuração de geração de texto definida
)

# Início de uma sessão de chat com o modelo
chat_session = model.start_chat(
  history=[
    # História de chat pode ser preenchida aqui, mas atualmente está vazia
  ]
)


In [6]:
# Inicialização de dicionário para armazenar sintomas das doenças e lista para erros
disease_symptoms = {}  # Dicionário para armazenar os sintomas associados a cada doença
disease_error = []  # Lista para armazenar erros ocorridos durante o processamento das doenças


In [7]:
def fetch_disease_info(disease):
    """
    Função para obter informações de uma doença a partir da URL da WHO.
    
    Parâmetros:
    disease (str): Nome da doença
    
    Retorna:
    response (requests.Response): Resposta da requisição HTTP
    ou
    None: Se ocorrer um erro na requisição
    """
    # Limpa e formata o nome da doença para criar a URL
    # Substitui espaços por hífens e remove caracteres não alfanuméricos e hífens
    disease_clear = re.sub(r"[^a-z0-9-]", "", disease.lower().replace(" ", "-"))
    disease_html = f'https://www.who.int/news-room/fact-sheets/detail/{disease_clear}'

    try:
        # Faz uma requisição HTTP para obter o conteúdo da página da doença
        response = requests.get(disease_html)
        # Verifica se a requisição foi bem-sucedida
        response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
    except requests.exceptions.RequestException as e:
        # Em caso de erro na requisição, adiciona o erro à lista e retorna None
        disease_error.append(e)
        return None
    
    # Aguarda 1 segundo para evitar sobrecarga no servidor
    time.sleep(1)
    return response


In [8]:
def parse_symptoms(response_content):
    """
    Função para analisar o conteúdo HTML e extrair os sintomas.

    Parâmetros:
    response_content (str): Conteúdo HTML da página da doença

    Retorna:
    symptoms (str): String contendo os sintomas extraídos
    """
    # Cria um objeto BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(response_content, 'html.parser')

    # Procura um cabeçalho <h2> que contenha as palavras 'consequences' ou 'symptoms'
    symptoms_header = soup.find('h2', string=re.compile(r'consequences|symptoms', re.IGNORECASE))
    
    # Se não encontrar um cabeçalho de sintomas, retorna uma string vazia
    if not symptoms_header:
        return ''

    # Inicializa uma string vazia para armazenar os sintomas
    symptoms = ''
    
    # Encontra o próximo elemento após o cabeçalho de sintomas
    current_element = symptoms_header.find_next_sibling()
    
    # Itera sobre os elementos subsequentes até encontrar outro cabeçalho <h2> ou o fim do documento
    while current_element and current_element.name != 'h2':
        # Adiciona o texto do elemento atual à string de sintomas
        symptoms += current_element.text
        # Passa para o próximo elemento
        current_element = current_element.find_next_sibling()
    
    # Retorna a string contendo os sintomas extraídos
    return symptoms


In [9]:
def generate_summary(symptoms):
    """
    Função para gerar um resumo dos sintomas usando um modelo generativo.

    Parâmetros:
    symptoms (str): String contendo os sintomas

    Retorna:
    symptoms_final (str): Resumo dos sintomas gerado pelo modelo
    """
    # Cria um prompt para o modelo generativo
    prompt = f'Summarize {symptoms} in just bulletpoints, without titles, about health symptoms with few words.'
    
    # Chama o modelo generativo para gerar um resumo
    response = model.generate_content(prompt)
    
    # Remove caracteres que não sejam letras ou espaços do resumo gerado
    symptoms_final = re.sub(r"[^a-zA-Z\s]", "", response.text)
    
    # Retorna o resumo dos sintomas final
    return symptoms_final



In [10]:
for disease in diseases:
    response = fetch_disease_info(disease)

    # Verifica se a resposta é válida (diferente de None ou False)
    if response and response.content:  # Verifica se há conteúdo na resposta
        symptoms = parse_symptoms(response.content)

        # Processa os sintomas apenas se não for uma string vazia
        if symptoms:  
            summary = generate_summary(symptoms)
            # Divide o resumo em uma lista de sintomas, removendo espaços em branco
            disease_symptoms[disease] = [item.strip() for item in summary.split('\n') if item.strip()]
        else:
            # Se não houver sintomas, armazena uma lista vazia
            disease_symptoms[disease] = []  
    else:
        # Se a resposta não for válida, armazena uma lista vazia
        disease_symptoms[disease] = []

    time.sleep(5)  # Aguarda 5 segundos para evitar sobrecarga no servidor


In [22]:
list(disease_symptoms.items())[55]

('Diabetes',
 ['Excessive thirst',
  'Frequent urination',
  'Blurred vision',
  'Fatigue',
  'Unexplained weight loss',
  'Heart attack',
  'Stroke',
  'Kidney failure',
  'Vision loss',
  'Foot problems'])

In [20]:
# Cria uma lista com as chaves do dicionário disease_symptoms cujos valores são listas vazias
empty_symptoms_keys = [key for key, value in disease_symptoms.items() if not value]

# Imprime o número de chaves com valores vazios
print(len(empty_symptoms_keys))
empty_symptoms_keys

151


['Adolescent and young adult health',
 'Adolescent pregnancy',
 'Ageing and health',
 'Alcohol',
 'Ambient (outdoor) air pollution',
 'Animal bites',
 'Antimicrobial resistance',
 'Arsenic',
 'Asbestos: elimination of asbestos-related diseases',
 'Assistive technology',
 'Autism',
 'Biodiversity and Health',
 'Blindness and vision impairment',
 'Blood safety and availability',
 'Brucellosis',
 'Burns',
 'Buruli ulcer (Mycobacterium ulcerans infection)',
 'Campylobacter',
 'Cancer',
 'Cardiovascular diseases (CVDs)',
 'Cervical cancer',
 'Chagas disease (also known as American trypanosomiasis)',
 'Child mortality (under 5 years)',
 'Childhood cancer',
 'Children: improving survival and well-being',
 'Children: new threats to health',
 'Chronic obstructive pulmonary disease (COPD)',
 'Civil registration and vital statistics',
 'Climate change',
 'Colorectal cancer',
 'Commercial determinants of health',
 'Community-based health insurance',
 'Condoms',
 'Congenital disorders',
 'Coronavir

In [27]:
# Filtrar doenças com sintomas não vazios
filtered_diseases_symptoms = {disease: symptoms for disease, symptoms in disease_symptoms.items() if symptoms}


In [28]:
# Contar o número de elementos no dicionário filtrado
num_elements = len(filtered_diseases_symptoms)
print("Número de elementos no dicionário filtrado:", num_elements)

Número de elementos no dicionário filtrado: 70


In [30]:
# Salvar o dicionário filtrado em um arquivo JSON
with open("filtered_diseases_symptoms_who.json", "w") as json_file:
    json.dump(filtered_diseases_symptoms, json_file, indent=4)